<a href="https://colab.research.google.com/github/MilizCom/Machine-Learning/blob/main/CNN_Pengidentifikasi_penyakit_pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
# Import libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Setelah mounting, Anda akan diminta untuk memberikan izin dan menyalin kode otorisasi.
# Masukkan kode otorisasi yang diberikan untuk mengakses Drive.

# Tentukan jalur dataset yang ada di Google Drive Anda
train_dir = '/content/drive/MyDrive/Pnumonia_dataset/chest_xray/train'
val_dir = '/content/drive/MyDrive/Pnumonia_dataset/chest_xray/val'
test_dir = '/content/drive/MyDrive/Pnumonia_dataset/chest_xray/test'

# Pastikan jalur tersebut sesuai dengan struktur folder di Google Drive Anda


In [ ]:
# Data Augmentation and Image Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Menggunakan input size 224x224 untuk VGG16
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Jangan shuffle agar urutan label tetap
)


In [ ]:
# 3. Fine-Tuning Model Pretrained (VGG16)
# Load VGG16 tanpa fully connected layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze semua layer di base model
for layer in base_model.layers:
    layer.trainable = False



In [ ]:

# Tambahkan layer custom untuk klasifikasi
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Output biner
])


In [ ]:

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks untuk mencegah overfitting dan tuning learning rate
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)


In [ ]:

# 4. Training Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    callbacks=[early_stopping, reduce_lr]
)


In [ ]:

# 5. Evaluasi Model
# Evaluate on test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")


In [ ]:

# Predict on test data
predictions = model.predict(test_generator)
predicted_labels = np.where(predictions > 0.5, 1, 0)  # Konversi probabilitas ke label biner
true_labels = test_generator.classes  # Label asli


In [ ]:

# Classification Report
report = classification_report(true_labels, predicted_labels, target_names=['NORMAL', 'PNEUMONIA'])
print("Classification Report:\n")
print(report)


In [ ]:

# 7. Plot Training and Validation Accuracy/Loss
plt.figure(figsize=(12, 6))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

batas
